In [4]:
!pip install kagglehub


In [8]:
# ============================================
# 1. Download Dataset From Kaggle with kagglehub
# ============================================
import kagglehub
import os

path = kagglehub.dataset_download("nazmul0087/ct-kidney-dataset-normal-cyst-tumor-and-stone")
print("Raw dataset folder:", path)

# --- Find folder containing classes ---
DATASET_DIR = None
for root, dirs, files in os.walk(path):
    if all(cls in dirs for cls in ["Cyst", "Normal", "Stone", "Tumor"]):
        DATASET_DIR = root
        break

if DATASET_DIR is None:
    raise Exception("❌ Folder kelas tidak ditemukan!")

print("✅ Dataset folder:", DATASET_DIR)

# ============================================
# 2. Libraries
# ============================================
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

IMG_SIZE = (96, 96)       # 🔥 sangat kecil, hemat RAM
BATCH_SIZE = 8            # 🔥 kecil agar training lebih ringan
EPOCHS = 3                # 🔥 cukup untuk model kecil
CLASS_NAMES = ["Cyst", "Normal", "Stone", "Tumor"]

# ============================================
# 3. Data Generator (TIDAK LOAD SEMUA DATA KE RAM)
# ============================================
datagen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.2,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

# ============================================
# 4. Build MobileNetV2 SUPER MINI
# ============================================
base_model = MobileNetV2(
    input_shape=(*IMG_SIZE, 3),
    include_top=False,
    weights="imagenet",
    alpha=0.35   # 🔥 sangat kecil, hemat RAM & ukuran model
)

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(32, activation='relu'),   # 🔥 kecil
    layers.Dropout(0.2),
    layers.Dense(4, activation='softmax')
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# ============================================
# 5. Train Model
# ============================================
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS
)

# ============================================
# 6. Save Model (sangat kecil)
# ============================================
os.makedirs("model", exist_ok=True)
model.save("model/kidney_model.h5")

print("🎉 Model kecil berhasil disimpan: model/kidney_model.h5")


Raw dataset folder: C:\Users\ASUS\.cache\kagglehub\datasets\nazmul0087\ct-kidney-dataset-normal-cyst-tumor-and-stone\versions\1
✅ Dataset folder: C:\Users\ASUS\.cache\kagglehub\datasets\nazmul0087\ct-kidney-dataset-normal-cyst-tumor-and-stone\versions\1\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone
Found 9959 images belonging to 4 classes.
Found 2487 images belonging to 4 classes.
2019640/2019640 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_0.35_96             │ (None, 3, 3, 1280)     │       410,208 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │        40,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 451,332 (1.72 MB)

 Trainable params: 41,124 (160.64 KB)

 Non-trainable params: 410,208 (1.56 MB)

Epoch 1/3
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 85s 65ms/step - accuracy: 0.7875 - loss: 0.5494 - val_accuracy: 0.5641 - val_loss: 1.1708
Epoch 2/3
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 71s 57ms/step - accuracy: 0.8878 - loss: 0.2947 - val_accuracy: 0.5798 - val_loss: 1.3596
Epoch 3/3
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 71s 57ms/step - accuracy: 0.9035 - loss: 0.2458 - val_accuracy: 0.5947 - val_loss: 1.3481


🎉 Model kecil berhasil disimpan: model/kidney_model.h5
